In [1]:
import sys

sys.path.append('..')

In [2]:
from src.encoders.encoders import SentenceEncoder, DRSEncoder

src_dir=PosixPath('/Users/edgar/Repositorios/LogicParserRL/src')
new_path=PosixPath('/data/encoder_data')
PATH_SENTENCE_ENCODER=PosixPath('../src/data/encoder_data/sentence_encoder.pth')


In [3]:
sentence_encoder = SentenceEncoder()
drs_encoder = DRSEncoder()

sentence_encoder.train(data=[])
drs_encoder.train(data=[])

x=tensor([[-0.8771, -1.1205,  1.1981, -0.5087, -0.5474, -0.3018, -0.4912,  0.2404,
          0.1648,  1.0735]])
out=tensor([[-0.4399,  0.0458, -0.3886, -0.1602, -0.1097,  0.1004,  0.3430, -0.1092,
          0.2418,  0.3189]], grad_fn=<AddmmBackward0>)
Model saved to ../src/data/encoder_data/sentence_encoder.pth
Model saved to /Users/edgar/Repositorios/LogicParserRL/src/../src/data/encoder_data/drs_encoder.pth
